In [12]:
import os
import time
import playsound
import time
import speech_recognition as sr
from gtts import gTTS
import numpy as np
import os
import urllib.request as ur
import ipywidgets as widgets

from openai import OpenAI
from IPython.display import IFrame, HTML
from io import BytesIO 
from PIL import Image
from IPython.display import display



key_API=os.environ['OPENAI_API']
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    # or you can explicitly pass in the key (NOT RECOMMENDED)
    api_key=key_API,
)

client

In [13]:

## Function which inputs speechtotext using gtts, prefer this from OpenAI due to not costing for the UI
def speak(text):
    tts = gTTS(text=text, lang="es")
    filename = "voice.mp3"
    tts.save(filename)
    playsound.playsound(filename)
    
#get the audio from the Mic to transform to text so it can interact with the prompts API of OpenAI
def get_audio(lang):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source,duration=0.2)
        audio = r.listen(source)
        said = ""

        try:
            said = r.recognize_google(audio,language=lang)
            print(said)
        except sr.UnknownValueError:
            print("No logre entenderte, ¿Puedes repetir eso?")
        except sr.RequestError as e:
            print(f"Error with the speech recognition service; {e}")
        #return None
        except Exception as e:
            print("Exception: " + str(e))

    return said

#Function used to record audio to be used to translate
def rec_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source,duration=0.2)
        audio = r.listen(source)
    said = r.recognize_google(audio,language="es-MX")
    print(said)
    tts = gTTS(text=said, lang="es")
    filename = "voice.mp3"
    tts.save(filename)
    #playsound.playsound(filename)
    return filename

#Fucntion used to create prompt and interact with dalee    
def dalee():
    speak("Que deseas que ponga en el diseño")
    logo_prompt= "Tema: "+get_audio("es-MX")
    speak("En que estilo")
    logo_prompt=logo_prompt+" Estilo:"+get_audio("es-MX") 

    image_size = 1024
    response = client.images.generate(
    model="dall-e-3",
    prompt=logo_prompt,
    size=f"{image_size}x{image_size}"
    )
    image_url = response.model_dump()['data'][0]['url']
    url = image_url
    with ur.urlopen(url) as url:
        img = Image.open(BytesIO(url.read()))
    display(img)
    print(image_url)
    

def translate():
    speak("Reproduce o di lo que deseas traducir")
    audio_file= open(rec_audio(), "rb")
    translation = client.audio.translations.create(
    model="whisper-1",
    file=audio_file
    )
    speak(translation.text)
    
    
def assistant():
    while True:
        speak("Como puedo asistirte?")
        #Translate speech to text
        SpeechText = get_audio()
        SpeechText = SpeechText.lower()
        
        #Manage if the user wants to leave the 
        if SpeechText=="salir":
            speak("Ten un buen día")
            break
        else:
            response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {
                "role": "system",
                "content": "Eres un util asistente"
                },
                {
                "role": "user",
                "content": "{}".format(SpeechText)
                }
                ]
            )

            # Print out results for further processing
            print(f"Human:{SpeechText}\nAI:{response.model_dump()['choices'][0]['message']['content']}")

            # SPEAK IT OUT
            speak(response.model_dump()['choices'][0]['message']['content'])
        continue
    

while True:

    try:

        with sr.Microphone() as source:

            #Greetings and present options
            speak("Buen día")
            #Translate speech to text
            SpeechText = get_audio("es-MX")
            SpeechText = SpeechText.lower()
            
            #Manage the option the user want to do
            if SpeechText=="salir":
                speak("Ten un buen día")
                break
            elif "hello" in SpeechText:
                speak("hello, how are you?")
            elif "what is your name" in SpeechText:
                speak("My name is Tim")
            elif "imagen" in SpeechText:
                dalee()
            elif "asistente" in SpeechText:
                assistant()
            elif "traductor" in SpeechText:
                translate()
            
                

           

    except sr.UnknownValueError:
       print("I didn't quite get you. Can you please repeat that?")
       speak("No logre entenderte, ¿Puedes repetir eso?")
       recognizer = sr.Recognizer()
    except sr.RequestError as e:
        print(f"Error with the speech recognition service; {e}")
    except Exception as e:
        print("Exception: " + str(e))
    continue

KeyboardInterrupt: 